In [3]:
import together
together.api_key = "c0a2a405a4779a5867a2c7ae4d83b6f0f24f7855b6f5e402296b40b1d51cc14a"

In [4]:
models = together.Models.list()

In [9]:
import together

import logging
import os
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

In [12]:
class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = together.api_key
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text

In [13]:
together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': '2e9d7aeb072954544e9edb047d9b340a08ee9b8c01604a732b78a372882a2743'}

In [14]:
test_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [17]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [18]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [31]:
instruction = "Chat History:\n\n{chat_history} \n\nHuman: {user_input}\n\n Assistant:"
system_prompt = "You are a helpful assistant and highly seasoned AI professional. You always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant and highly seasoned AI professional. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

Human: {user_input}

 Assistant:[/INST]


In [32]:
prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [33]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [34]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [35]:
llm_chain.predict(user_input="Hi, my name is Sinisa")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant and highly seasoned AI professional. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

Human: Hi, my name is Sinisa

 Assistant:[/INST]

> Finished chain.


" Hello Sinisa, it's nice to meet you. How can I assist you today?"

In [36]:
llm_chain.predict(user_input='''
Monica is a data scientist at a large grocery store chain.

The grocery store has modern cash register systems, so Monica can access a wealth of sales data. She's fascinated by the massive variety of items customers purchase every day. The store sells everything from fruits and vegetables to household items.

Monica is thinking about creating customer profiles to understand purchasing patterns better. This might help the store to design better sales promotions or arrange products more efficiently. However, she's unsure whether she should segment customers based on the types of products they buy, their total spending, or the time of their purchases.

If you were in Monica's position, how would you approach this problem?

1. Define a few profiles beforehand, and train a supervised learning algorithm to classify every customer into one.
2. Use a supervised learning algorithm to produce potentially interesting ways to segment the customers.
3. Use an unsupervised learning algorithm to produce potentially interesting ways to segment the customers.
4. Use a semi-supervised learning algorithm to process the data, thus taking advantage of both supervised and unsupervised techniques.
''')



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant and highly seasoned AI professional. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sinisa
AI:  Hello Sinisa, it's nice to meet you. How can I assist you today? 

Human: 
Monica is a data scientist at a large grocery store chain.

The grocery store has modern cash register systems, so Monica can access a wealth of sales data. She's fascinated by the massive variety of items customers purchase every day. The store sells everything from fruits and vegetables to household items.

Monica is thinking about creating customer profiles to understand purchasing patterns better. This might help the store to design better sales promotions or arrange products more efficiently. However, she's unsure whether she should segment customers based on the types of products they buy, their total spending, or the ti

" Hello! I'm just an AI, I can provide you with some suggestions based on my training data. If I were in Monica's position, I would approach this problem by using a combination of unsupervised and supervised learning algorithms.\n\nFirst, I would start by using an unsupervised learning algorithm, such as k-means clustering or hierarchical clustering, to group customers based on their purchasing patterns. This would help me identify natural groupings or clusters of customers without any prior knowledge of the number of profiles or segments.\n\nNext, I would use a supervised learning algorithm, such as logistic regression or decision trees, to classify each customer into one of the clusters identified in the previous step. This would help me understand the characteristics of each cluster and their corresponding purchasing patterns.\n\nFinally, I would use the clusters and their characteristics to design better sales promotions or arrange products more efficiently. For example, if a clust

In [37]:
llm_chain.predict(user_input="So, based on your previous answer, which one from the 4 given sentences would you choose?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant and highly seasoned AI professional. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sinisa
AI:  Hello Sinisa, it's nice to meet you. How can I assist you today?
Human: 
Monica is a data scientist at a large grocery store chain.

The grocery store has modern cash register systems, so Monica can access a wealth of sales data. She's fascinated by the massive variety of items customers purchase every day. The store sells everything from fruits and vegetables to household items.

Monica is thinking about creating customer profiles to understand purchasing patterns better. This might help the store to design better sales promotions or arrange products more efficiently. However, she's unsure whether she should segment customers based on the types of products they buy, their total spending, or the time

' Based on my previous answer, I would choose option 3: Use an unsupervised learning algorithm to produce potentially interesting ways to segment the customers. This is because unsupervised learning algorithms, such as k-means clustering or hierarchical clustering, can help identify natural groupings or clusters of customers without any prior knowledge of the number of profiles or segments. Additionally, using unsupervised learning algorithms can help identify patterns or structures in the data that may not be immediately apparent, which can lead to new insights and discoveries.'

In [38]:
together.Models.stop("togethercomputer/llama-2-70b-chat")

{'success': True}